# Lecture 3: Stream processing with Spark

_Spark Structured Streaming_ is an extension of the Spark API that enables scalable stream processing.

It's a well-documented API [link](https://spark.apache.org/docs/3.5.0/structured-streaming-programming-guide.html), which is however different from Spark Streaming, which has recently become deprecated since Spark version 3.4.0 (~last year)

The main difference between Spark `Streaming` and `Structured Streaming` is that the former is based on the low-level RDD API, while the latter is based on DataFrames.

Both APIs have however been designed to help with the continuous processing of streaming applications. 

The continuous stream of input data can be ingested from many data sources such as **Kafka**, **Amazon S3**, or **TCP sockets**. 

Processed data can be exported to an external database and used to make live dashboards or offline analyses, stored in files, or used in a further stage of a Kafka pipeline. 

Overall, the practice of reading data from a set of sources, pre-processing it, and then storing it in a different format for later analysis is extremely common and has its own name: **real-time ETL pipelines**.
- **E**xtract
- **T**ransform
- **L**oad


## Structured Streaming

The key idea of this stream processing model is to treat the continuous stream as a table that is continuously appended to.

This allows users to view the continuously incoming data as a DataFrame with new records being new rows to be included. It further expresses stream processing as a standard batch-like query on a static table, similar to what we have done in the Spark DataFrame lecture.

![](imgs/lecture3/structured-streaming-stream-as-a-table.png)

Internally, the stream is divided into micro-batches produced by a "trigger," which could represent any given condition (e.g., every 1 second):

- The input stream is a table (DataFrame) or the new rows to be appended to the previous DataFrame.
- Every operation/query will produce a result table (DataFrame).
- When the output table is updated, it can be written somewhere thanks to an output module/type.

![](imgs/lecture3/structured-streaming-model.png)

**NB:** Although the input table can be viewed as an always-growing DataFrame, **Spark does not actually materialize the entire table**. 

- Only the latest data are processed (latest batch) and then discarded.
- Conversely, the result table can be updated to keep track of the results from previous batches.

There are three output modes:

- ***Complete Mode*** where the entire output table is written to the Sink.
- ***Append Mode*** where only the new rows appended in the Result Table since the last trigger will be processed by the Sink.
- ***Update Mode*** where only the rows that were updated in the result table since the last trigger will be written to the Sink.

In this notebook, the *Update Mode* and *Console* output sink will be used.

- Results (i.e., only the updated records) will be displayed on the screen (i.e., on the "terminal").
- However, the Sink could be a database, a file, Kafka, or more, depending on the needs and applications.


## Create and Start a Spark Session

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("My streaming spark application")\
    .config("spark.executor.memory", "512m")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
    .config("spark.sql.adaptive.enabled", "false")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 07:02:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

### TCP Socket Source

For this example, Spark will read data from a TCP socket using Spark Structured Streaming.

A TCP socket is a communication endpoint used to establish a connection between two devices over a network. You can think of it as a telephone connection: two endpoints have to establish a connection; once the connection is established, communication can occur, with data transfer; as soon as one of the two ends interrupts the connection, the whole communication is lost. 

We will generate a dummy data stream representing fake credit card transactions.

A simple Python program will be used to create this data stream. You will be able to find it in `utils/producer.py`. When executed, the producer will try to establish a TCP connection and send data on port `5555` of a given `host` (`spark-master` in our case). 

Before executing the producer program, take a moment to review the `producer.py` code to understand how it works. It's important to understand the logic of the program before using it to generate the streaming data.


In [3]:
! cat utils/producer.py

import socket
import json
import time
import random
import argparse

# Define some lists of first and last names to use for generating random messages
first_names=('John','Andy','Joe','Alice','Jill')
last_names=('Johnson','Smith','Jones', 'Millers','Darby')

# Define a function for sending messages over the socket
def send_messages(client_socket):
    try:
        while 1:
            # Generate a random message with a random name, surname, amount, delta_t, and flag
            msg = {
                'name': random.choice(first_names),
                'surname': random.choice(last_names),
                'amount': float('{:.2f}'.format(random.random()*1000)),
                'delta_t': float('{:.2f}'.format(random.random()*10)),
                'flag': int(random.choices([0,1], weights=[0.8, 0.2])[0])
            }
            try:
              # Encode the message as JSON and send it over the socket
              client_socket.send((json.dumps(msg)+"\n").encode('utf-8'))
           

The producer will generate new records in the form of a random combination of:
- `name`
- `surname`
- `amount`: amount of the credit card transaction
- `delta_t`: time between transactions
- `flag`: random flag to indicate if potentially fraudulent or not

This information will be formatted into a `.json` data format

### Creating the streaming DataFrame from a TCP socket source

To inform Spark that the data source will be a TCP socket located at a specific `hostname` and `port`, we can use the options `host` and `port` methods.
* When declaring this source a message appears, warning that the TCP source is only meant for testing purposes.

Refer to the [documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamReader.html) for additional available options.

Notice that the syntax is equivalent to the one used for example to read a set of files from a disk.


In [4]:
# the hostname and port number
hostname = "spark-master"
portnumber = 5555

rawMessagesDf = (
    spark
    .readStream
    .format("socket")
    .option("host", hostname)
    .option("port", portnumber)
    .load()
)

24/05/16 07:02:29 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


### Start the python producer.py script

From a terminal/WSL, connect to the `spark-master` Docker container using the command
```bash
docker exec -it spark-master bash
``` 

From inside the docker container, move to the `/mapd-workspace` folder and execute the python script with the option `--hostname spark-master`:

```bash
python /mapd-workspace/notebooks/utils/producer.py --hostname spark-master
```

The producer application will be automatically closed when the streaming application terminates.

### Running the first streaming application

The first streaming query will be a simple `show` of the DataFrame
* No processing between `rawMessagesDf` and output sink (`writeStream`)

Output mode is set to `update`, hence every new message received by the TCP source will be processed by the output sink.
* Output format `console` indicates that the output will be displayed on the screen before being discarded

The streaming query is triggered every 2 seconds
* refer to the [documentation](https://spark.apache.org/docs/3.5.0/structured-streaming-programming-guide.html#streaming-table-apis) for additional trigger types.

In [5]:
query = (
    rawMessagesDf
    .writeStream
    .outputMode("update")
    .format("console")
    .trigger(processingTime='2 seconds')
    .option("truncate", False)
    .start()
)

24/05/16 07:02:30 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-927310b2-f240-4c11-a1e4-86bc7209a4f7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Run this cell to stop the streaming query execution

In [6]:
query.stop()

## Example of streaming query: data parsing

Data received from the TCP source is seen as a `string` by the spark application

We first must develop an application parsing the string and creating a column for each `json` field in order to start processing the dataset using the DataFrame API functionalities

It can be useful to start with a set of test data, to develop this query.

**The same Spark code (transformations/actions) used for processing a "static" DataFrame can be used for the streaming context!**

In [7]:
# dummy data for testing purposes
testData = [
    '{"name": "Jill", "surname": "Millers", "amount": 736.56, "delta_t": 7.78, "flag": 0}',
    '{"name": "John", "surname": "Johnson", "amount": 986.47, "delta_t": 3.9, "flag": 0}',
    '{"name": "John", "surname": "Jones", "amount": 249.9, "delta_t": 0.62, "flag": 1}',
    '{"name": "Andy", "surname": "Jones", "amount": 950.95, "delta_t": 6.02, "flag": 0}',
    '{"name": "Jill", "surname": "Millers", "amount": 724.32, "delta_t": 9.19, "flag": 0}',
    '{"name": "John", "surname": "Johnson", "amount": 850.07, "delta_t": 7.33, "flag": 1}',
    '{"name": "Andy", "surname": "Smith", "amount": 557.48, "delta_t": 9.64, "flag": 0}',
    '{"name": "Alice", "surname": "Darby", "amount": 424.75, "delta_t": 7.76, "flag": 0}'
]

Create a Spark DataFrame by importing the `testData`.

It can be useful to investigate the `pyspark.sql.types` to check whether there is any helping function that could be used for this purpose.

In [8]:
from pyspark.sql.types import StringType

# create a spark dataframe from testdata
testDf = spark.createDataFrame(testData, StringType())

In [9]:
# show the dataframe content
testDf.show(n=5, truncate=False)

+------------------------------------------------------------------------------------+
|value                                                                               |
+------------------------------------------------------------------------------------+
|{"name": "Jill", "surname": "Millers", "amount": 736.56, "delta_t": 7.78, "flag": 0}|
|{"name": "John", "surname": "Johnson", "amount": 986.47, "delta_t": 3.9, "flag": 0} |
|{"name": "John", "surname": "Jones", "amount": 249.9, "delta_t": 0.62, "flag": 1}   |
|{"name": "Andy", "surname": "Jones", "amount": 950.95, "delta_t": 6.02, "flag": 0}  |
|{"name": "Jill", "surname": "Millers", "amount": 724.32, "delta_t": 9.19, "flag": 0}|
+------------------------------------------------------------------------------------+
only showing top 5 rows



Data are now in the same format as the one received by the socket data source. 

The function `from_json` can be used to parse a `json` string with a given schema. Aa always, have a look at the [documentation](https://spark.apache.org/docs/3.5.0/api/python/reference/pyspark.sql/api/pyspark.sql.functions.from_json.html?highlight=from_json) before using it.

Start by defining a schema for our data.

In [10]:
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType

# create the schema
schema = StructType(
    [
        StructField("name", StringType()),
        StructField("surname", StringType()),
        StructField("amount", DoubleType()),
        StructField("delta_t", DoubleType()),
        StructField("flag", IntegerType()),
    ]
)

And create a "parsed" DataFrame to verify if our schema is properly addressing the data types.

In [12]:
# parse the dataframe and show its structure
parsedTestDf = testDf.select(f.from_json('value', schema=schema))

parsedTestDf.printSchema()

root
 |-- from_json(value): struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- surname: string (nullable = true)
 |    |-- amount: double (nullable = true)
 |    |-- delta_t: double (nullable = true)
 |    |-- flag: integer (nullable = true)



The structure obtained using `from_json` is actually inherently nested...
The `json` messages were correctly parsed, but a nested DataFrame is returned, which should be flattened before being able to use it as a "plain DataFrame".


As discussed in the DataFrame notebook, this can be easily solved in a variety of ways, for instance by selecting the columns we are interested in using from now on...


As an example, the `name` and `surname` columns could be selected.

In [13]:
# show values from the parsedTestDf
parsedTestDf.show(n=5, truncate=False)

+--------------------------------+
|from_json(value)                |
+--------------------------------+
|{Jill, Millers, 736.56, 7.78, 0}|
|{John, Johnson, 986.47, 3.9, 0} |
|{John, Jones, 249.9, 0.62, 1}   |
|{Andy, Jones, 950.95, 6.02, 0}  |
|{Jill, Millers, 724.32, 9.19, 0}|
+--------------------------------+
only showing top 5 rows



In [14]:
# re-create the parsedTestDf by 
# - aliasing the values produced by from_json as `data`
# - selecting the columns `data.name` and `data.surname`
parsedTestDf = (
    testDf
    .select(f.from_json('value', schema=schema).alias('data'))
    .select(f.col('data.name'), f.col('data.surname'))
)

# NB: data.* will select all the columns, as in plain SQL

In [15]:
# print the new schema
parsedTestDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)



In [18]:
# and show the new content
parsedTestDf.show(5)

+----+-------+
|name|surname|
+----+-------+
|Jill|Millers|
|John|Johnson|
|John|  Jones|
|Andy|  Jones|
|Jill|Millers|
+----+-------+
only showing top 5 rows



### From static to streaming queries

Now that we have developed a way to extract and interpret the data using a static example, we can reuse the very same code for the streaming query, by simply "chaining" the application to the source and directing its results to the sink.

**NB**: Remember to restart the producer application before starting the queries.

Re-create the "raw" DataFrame connecting the Spark Structured Streaming Context to the input TCP socket.

**Do not `start` the stream just yet!**

In [16]:
# the hostname and port number
hostname = "spark-master"
portnumber = 5555

# recreate the streaming raw dataframe
rawMessagesDf = (
    spark
    .readStream
    .format("socket")
    .option("host", hostname)
    .option("port", portnumber)
    .load()
)

24/05/16 07:03:27 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


Starting from the streaming DataFrame, issue the appropriate transformations to interpret its `json` format, and prepare a new parsed DataFrame including all (`*`) the columns.

In [ ]:
# parse the json lines using the previous schema
# select all columns 
parsedDf = (
    rawMessagesDf."""..."""
 
)

Start the application as done previously, with:
- `outputMode=update`
- `format=console`

Choose the trigger as you prefer. 
A suggestion is to keep it simple, and use a time-based trigger of 2 seconds.

In [ ]:
query = (
    parsedDf
    .writeStream
    """..."""
    """..."""
    """..."""
    .option("truncate", False)
    .start()
)

In [ ]:
query.stop()

### Process each bach to identify possibly fraudolent transactions

1. compute the _number of flagged transactions per batch per user_ (create a unique `userID` field as the combination of _FirstLastname_ to idenfity individual users)
2. identify all the "suspicios" transactions per user: all users with more than one flagged transaction per batch will be assigned a `isFraud` boolean variable
3. format the resulting `userID` and `isFraud` information in a DataFrame to mimick a "live-report" of the suspicious transactions

In [ ]:
# the hostname and port number
hostname = "spark-master"
portnumber = 5555

rawMessagesDf = 

In [ ]:
# parse json lines, use a schema
parsedDf = (
    rawMessagesDf.

    
)


In [ ]:
# find number of transactions for each user when flag = 1 
#    declare a new column to create a unique user identifier 
#    this can be easily done by concatenating first- and last-name fields
#    check the concat function from pyspark.sql.functions 
numTransactions = (
    parsedDf
 
 
) 

In [ ]:
# find suspicious transactions
#    filter only users with more than one transaction per batch
#    create a "fraud" column with a value of 1 for the selected users (check the lit function)
#    from the dataframe, project the unique id, fraud flag and number of transaction columns
susTransactions = (
    numTransactions


)                                   

In [ ]:
# this line is a trick to force Spark to use a small number of partitions (4 in this example)
spark.conf.set("spark.sql.shuffle.partitions", 4)

In [ ]:
query = (
    susTransactions
    """
    ...
    ...
    ...
    """
    .start()
)

In [ ]:
query.stop()

Since this query contains aggregations, output mode can be changed to `complete`.
In this case, the full set of results is displayed for every batch. 

In [ ]:
query = (
    susTransactions
    """
    ...
    ...
    ...
    """
    .start()
)

In [ ]:
query.stop()

## Stop spark worker

In [ ]:
spark.stop()